In [24]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel
from typing import List
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../.env")

llm = init_chat_model("ollama:gpt-oss:120b", base_url=os.environ.get("OLLAMA_BASE_URL"))


class State(TypedDict):
    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str


class Ingredient(BaseModel):
    name: str
    quantity: str
    unit: str


class IngredientsOutput(BaseModel):
    ingredients: List[Ingredient]

In [25]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    response = structured_llm.invoke(
        f"List 5-8 ingredients needed to make {state["dish"]}"
    )

    return {"ingredients": response.ingredients}


def create_recipe(state: State):
    response = llm.invoke(
        f"Write a step by step cooking instruction for {state["dish"]} using these ingredients {state["ingredients"]}"
    )

    return {
        "recipe_steps": response.content,
    }


def describe_plating(state: State):
    response = llm.invoke(
        f"Describe how to beautifully plate this {state["dish"]} based on this recipe {state["recipe_steps"]}"
    )

    return {
        "plating_instructions": response.content,
    }

In [26]:
graph_builder = StateGraph(State)

graph_builder.add_node("list_ingredients", list_ingredients)
graph_builder.add_node("create_recipe", create_recipe)
graph_builder.add_node("describe_plating", describe_plating)

graph_builder.add_edge(START, "list_ingredients")
graph_builder.add_edge("list_ingredients", "create_recipe")
graph_builder.add_edge("create_recipe", "describe_plating")
graph_builder.add_edge("describe_plating", END)


graph = graph_builder.compile()


graph.invoke({"dish": "hummus"})

{'dish': 'hummus',
 'ingredients': [Ingredient(name='Cooked chickpeas (garbanzo beans) – 1½ cups (or 1 can, drained)', quantity='≈ 400\u202fg', unit='grams'),
  Ingredient(name='Tahini (sesame‑seed paste)', quantity='¼\u202fcup', unit='cup'),
  Ingredient(name='Fresh lemon juice', quantity='2\u202ftablespoons', unit='tbsp'),
  Ingredient(name='Extra‑virgin olive oil', quantity='2\u202ftablespoons', unit='tbsp'),
  Ingredient(name='Garlic, minced', quantity='1\u202fclove', unit='clove'),
  Ingredient(name='Sea salt', quantity='½\u202fteaspoon', unit='tsp'),
  Ingredient(name='Ground cumin (optional)', quantity='¼\u202fteaspoon', unit='tsp'),
  Ingredient(name='Water (or reserved chickpea liquid)', quantity='2‑4\u202ftablespoons', unit='tbsp')],
 'recipe_steps': '## Classic Hummus – Step‑by‑Step\n\n| Ingredient | Amount |\n|------------|--------|\n| Cooked chickpeas (garbanzo beans) – 1½\u202fcups (≈\u202f400\u202fg) or 1\u202fcan, drained | 1½\u202fcups |\n| Tahini (sesame‑seed paste) |